In [1]:
import os                                                                          
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"                                       
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import tensorflow as tf
session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.1, allow_growth=True), allow_soft_placement=True, log_device_placement=False)
sess = tf.Session(config=session_config)

from tensorflow.contrib.slim.nets import resnet_v2, resnet_utils
import tensorflow as tf
from tensorflow.contrib import layers as layers_lib
from tensorflow.python.ops import variable_scope
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.contrib import slim
from tensorflow.nn import ctc_loss, conv2d
import numpy as np
resnet_v2_block = resnet_v2.resnet_v2_block
resnet_v2 = resnet_v2.resnet_v2

import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def resnet_v2_26_base(inputs,
                 num_classes=None,
                 is_training=True, # True - due to update batchnorm layers
                 global_pool=False,
                 output_stride=1, # effective stride 
                 reuse=None,
                 include_root_block=False, #first conv layer. Removed due to max pool supression. We need large receprive field
                 scope='resnet_v2_26'):
  
    """
    Tensorflow resnet_v2 use only bottleneck blocks (consist of 3 layers).
    Thus, this resnet layer model consist of 26 layers.
    I put stride = 2 on each block due to increase receptive field.

    """
    blocks = [
      resnet_v2_block('block1', base_depth=64, num_units=2, stride=2),
      resnet_v2_block('block2', base_depth=128, num_units=2, stride=2),
      resnet_v2_block('block3', base_depth=256, num_units=2, stride=2),
      resnet_v2_block('block4', base_depth=512, num_units=2, stride=2),
    ]
    return resnet_v2(
      inputs,
      blocks,
      num_classes,
      is_training,
      global_pool,
      output_stride,
      include_root_block,
      reuse=reuse,
      scope=scope)

def make_ocr_net(inputs, num_classes, is_training=True):
    '''
    Creates neural network graph.
    Image width halved and it's define timestamps width (feature sequence length) 
    No activation after output (no softmax), due to it's presence at ctc_loss() and beam_search().
    After resnet head features are resized to be [batch,1,width,channel], and after that goes 1x1 conv 
    to make anology of dense connaction for each timestamp.
    
    input: batch of images
    output: tensor of size [batch, time_stamps_width, num_classes]
    '''
    with tf.variable_scope('resnet_base', values=[inputs]) as sc:
        with slim.arg_scope([slim.conv2d],
                              activation_fn=None, normalizer_fn=None):
            net = resnet_utils.conv2d_same(inputs, 64, 7, stride=2, scope='conv1') #root conv for resnet
            #net = slim.max_pool2d(net, [3, 3], stride=2, scope='pool1') # due to enlarge of receptive field
            net = resnet_v2_26_base(net, output_stride=1, is_training = is_training)[0] # ouput is a tuple of last tensor and all tensors 
    with tf.variable_scope('class_head', values=[net]) as sc:
        net = tf.transpose(net, [0,3,1,2]) # next 4 lines due to column to channel reshape. [batch,c,h,w]
        _,c,h,_ = net.get_shape() # depth of input to conv op tensor should be static (defined)
        shape = tf.shape(net)
        net = tf.reshape(net, [shape[0], c*h, 1, shape[3]])
        net = tf.transpose(net,[0,2,3,1]) # back to [batch,h,w,c] = [batch,1,w,features*h]
        net = layers_lib.conv2d(net, num_classes, [1, 1], activation_fn=None) #CTC got softmax [batch,1,w,num_classes]
        net = tf.squeeze(net,1) #[batch,w,num_classes]
        return net

def get_prediction(output_net, seq_len, merge_repeated=False):
    '''
    predict by using beam search
    input: output_net - logits (without softmax) of net
           seq_len - length of predicted sequence 
    '''
    net = tf.transpose(output_net, [1, 0, 2]) #transpose to [time, batch, logits]
    decoded, prob = tf.nn.ctc_beam_search_decoder(net, seq_len, merge_repeated=merge_repeated)
    return decoded, prob


In [3]:
with open('../../data_generator/valid_charset.txt', 'r', encoding = 'utf-8') as f:
    valid_charset = f.read()
all_chars = list(valid_charset)
char_to_indx = dict(zip(all_chars,range(len(all_chars))))
num_classes = len(all_chars)

In [4]:
class OCRModel(object):
    def __init__(self, charset, input_image_batch, sequence_labels, is_training=True, learning_rate=1e-4,
                decay_steps=2**16, decay_rate=0.9, decay_staircase=False, momentum=0.9):
        self.charset = charset
        self.num_classes = len(charset)+2 #indexing starts from one, +empty char. Yes, I know that I got 0-th rudimental output, whatever
        self.is_training = is_training
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate 
        self.decay_staircase = decay_staircase
        self.momentum = momentum
        mapping_char = tf.constant(list(charset))
        self.table = tf.contrib.lookup.index_to_string_table_from_tensor(
                    mapping_char, default_value="blank")
        self.build_for_prediction(input_image_batch)
        
    def build_for_prediction(self, input_image_batch):
        
        self.input_image_batch = input_image_batch
        net = make_ocr_net(self.input_image_batch, self.num_classes, is_training=self.is_training)
        self.net = net
        self.prediction = get_prediction(net, [tf.shape(self.net)[1]], merge_repeated=False) # tuple(decoded, prob). decoded - list of top paths. I use top1
        pred_dense = tf.sparse_to_dense(self.prediction[0][0].indices, self.prediction[0][0].dense_shape, 
                                               self.prediction[0][0].values)
        self.pred_dense = pred_dense
        self.prediction_string = tf.reduce_join(self.table.lookup(tf.cast(pred_dense, tf.int64)-1))
        

In [5]:
import json
from skimage import io
from skimage.transform import rescale
from skimage.color import rgb2gray
def resize_height(image, max_dim):
    h, w = image.shape
    scale_factor = max_dim/h
    output_image = rescale(image.copy(), scale = scale_factor, order = 3)
    return output_image

In [6]:
graph = tf.Graph()
with graph.as_default():

    features = tf.placeholder(tf.float32, shape = [None, 32, None, 1])
    model = OCRModel(charset = all_chars, input_image_batch=features, 
                     sequence_labels=None, is_training=False)
    init = tf.global_variables_initializer()
    table_init = tf.tables_initializer()
    saver = tf.train.Saver()

In [7]:
from lev_dist import distance

In [8]:
LOG_DIR = 'log/'
with tf.Session(graph=graph) as sess:
#     run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#     run_metadata = tf.RunMetadata()
    sess.run([init, table_init])
    try:
        saver.restore(sess, LOG_DIR + "model_lev_dist_0.2719941635845625.ckpt")
    except:
        print('cant restore')
    with open('../datasets/labeled_strings.json', 'r') as f:
        string_data = json.load(f)
    match = 0
    lev_dist = 0
    for example in string_data:
        image = io.imread('../datasets/' + example['path'])
        image = rgb2gray(image)
        image = resize_height(image, 32)
#         image = image/255 #as image scaled to [0,1]
        image = image - 0.5
        string = example['context']
        pred_string = sess.run([model.prediction_string], feed_dict={features: image[np.newaxis,...,np.newaxis]})
        pred_string = pred_string[0].decode('utf-8')
        print(pred_string.upper(), string)
        lev_dist += distance(pred_string.upper(), string) 
        if pred_string.upper() == string:
            match+=1

INFO:tensorflow:Restoring parameters from log/model_lev_dist_0.2719941635845625.ckpt


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


ШWИМ №М РИС ДЛИННОЗЕРНЫЙ 800Г ЯРМАРКА
DAT :П=0=ФK Data : 11-08-2017
РУБ РУБ
89 89
78 78
1 КГ=112,23PYБ 1 КГ = 112,23РУБ
PИC БЕЛЬИ ДЛИНО3ЕPHЫИ 9 рис белый длинозерный 900г
MИСTРAЛЬ OРИEHТ МИСТРАЛЬ ОРИЕНТ
DNT~Q2O DATA :27-10-2017
4 KГ=Я9 R9NVБ 1 кг = 82.89 руб
PУБ РУБ
60| 60
74 74
PИC 3OЛOTИСТЫИ 9ЮГ AHГСTPEM РИС ЗОЛОТИСТЫЙ 900Г АНГСТРЕМ
DАTА :ЭO-08~2OL7 DATA :30-08-2017
 KГ =1 77NVБ 1 КГ =100.77 РУБ
РУБ РУБ
69 69
90 90
PУБ РУБ
33 33
61 61
1 КГ"*48.90РУП 1 КГ =45.90РУБ
РУБ РУБ
90 90
45 45
/C ДWИJPHЫИ А3А1СXИ РИС ДЛИННОЗЕРНЫЙ АЗИАТСКИЙ
ПАСCИM ПАССИМ
ATЯ \№0=10=H DATA : 30-10-2017
1 КГ=115,88РУБ 1 КГ =115,88РУБ
РУБ РУБ
94 94
57 57
1 КГ=63 90NVБ 1 КГ =63.90 РУБ
PУБ РУБ
90| 90
63 63
1 KГ=77,80PY0 1 КГ =77,80РУБ
РУБ| РУБ
90 90
38 38
1 КГ=153,ЗJPУБ 1 КГ =153,33РУБ
РУБ| РУБ
33| 33
6 61
1 KГ=77,80PУБ 1 КГ =77,80РУБ
PУБ РУБ
90 90
38 38
1 КГ=153,33PУБ 1 КГ =153,33РУБ
PУБ РУБ
33| 33
61 61
СУПБOБO8ЫИ 25Г ЯPMPK СУП БОБОВЫЙ 250Г ЯРМАРКА
DATА :11~1O-2O17 DATA :11-10-2017
22 122
00) 00
PYБ РУБ
1 KГ=488,O

1КГ=75.80PУБ 1 КГ =75.80 РУБ
PYБ РУБ
90| 90
3 37
1 KГ=130,34PУБ 1 КГ =130,34РУБ
PУБ РУБ
17 1 7
65 65
1KГ=56 S0РУБ 1 КГ=56.90РУБ
PУБ РУБ
90 90
56 56
DATВ :1L1~OB-2O17 DATA :11-08-1017
1 KГ=42 23NVБ 1 КГ =112.23РУБ
РУБ РУБ
78 78
89 89
PИС ДЛЯ ПЛOВА 900ГР РИС ДЛЯ ПЛОВА 900ГР
DATА :O4-O2-2O17 DATA :04-02-2017
1 КГ=91,29РУБ 1 КГ =91,29РУБ
РУБ РУБ
16| 16
82 82
СУПБОБОВЫИ 25Г ЯPМAPKА СУП БОБОВЫЙ 250Г ЯРМАРКА
2 122
00 00
PУБ РУБ
1 KГ=488,00PYБ 1 КГ =488,00РУБ
DАTA :1L-1O-2O17 DATA :11-10-2017
PИС 5X100ГР ДOHГУСTО МDD| РИС 5X100 ДОН ГУСТО MDD
DАTА :19-09-2D17 DATA :19-09-2017
1 KГ =75,80PYБ 1 КГ=75,80РУБ 
РУБ РУБ
90| 90
37 37
РИС ДЛЯ ПЛOBА 900ГР РИС ДЛЯ ПЛОВА 900ГР
DATAO2~№ DATA :04-02-2017
1 КГ =91,29РУБ 1 КГ =91,29РУБ
РYБ РУБ
16 1 6
82 82
PИС 5Х100ГP ДOH ГУСTО МDO РИС 5Х100ГР ДОН ГУСТО МDD
DЗTИ :19-O9~2Q17 DATA :19-09-2017
1 КГ =75,80РУБ 1 КГ =75,80РУБ
РУБ РУБ
90 90
3 37
PИС"ЭKCTPА"ЮУБAHСХИИ 3ЗПИTHЫИ HBC "ЭКСТРА" КУБАНСКИЙ ЭЛИТНЫЙ
900Г 900Г
DАTA :O4-O8-2O17 DATA :04-08-2017
1КГ=89,78PУБ 1 КГ 

РУC РУБ
80 80
80 80
PИС KУБАHСKИИ 9O0ГР РИС КУБАНСКИИ 900ГР
DАTА :28-LO-2O17 DATA :28-10-2017
1 KГ =79,77PУБ 1 КГ =79,77РУБ
РУБ РУБ
79 79
1 7 1
PПЫИДOЖПCT0 1MO РИС КРУГЛЫЙ ДОН ГУСТО 1КГ MDD
DATA :2O-1O-2O17 DATA :20-10-2017
1КГ =56,90PУБ 1 КГ =56,90РУБ
PУБ РУБ
90 90
56 56
ПACСИM ПАССИМ
1 KГ=115,BBРУБ| 1 КГ =115,88РУБ
РУБ РУБ
94 94
5 57
PЮУГПЫИПРVMЮRСXИ5 РИС КРУГЛЫЙ ПРИМОРСКИЙ 500Г
ПАССИM ПАССИМ
DАTA :22-1O-2OI7 DATA :22-10-2017
КГ=115,88PУБ 1 КГ =115,88РУБ
PУБ РУБ
94 94
57 57
 KГ =56,90РУБ 1 КГ = 56,90РУБ
PУБ РУБ
90| 90
56 56
PИС KPУГЛЫИ ПРИМOРСKИИ 5ЮГ РИС КРУГЛЫЙ ПРИМОРСКИЙ 500Г
ПAССИМ ПАССИМ
DАTА :22-LO~2O1 DATA :22-10-2017
1 КГ=115,88PУБ 1 КГ =115,88РУБ
PУБ РУБ
94 94
5 57
PШЯKAШИ8ГАГPOЮПЬTPA РИС ДЛЯ КАШИ 800Г АГРОКУЛЬТУРА
DATA :O8-IО-2O17 DATA :08-10-2017
1КГ= 1 КГ =40.16РУБ
PУБ РУБ
13 13
32 32
ФAСОЛЬ 800Г ШЕБEХИHСKАЯ ФАСОЛЬ 800Г ШЕБЕКИНСКАЯ
DATA :26-O1-2O17 DATA :26-01-2017
1 КГ=122,6OPУБ 1 КГ =122,60РУБ
PУБ РУБ
08| 08
98 98
PИСKPУГЛO3EРHЫИ 1KГKД РИС КРУГЛОЗЕРНЫЙ 1КГ КД
DATA :28-1O

...

1OOГР= 329* 100 ГР = 32940
10O ГР=388" 100 ГР =38888
10O ГР=3O2" 100ГР = 30206
1KГ = 1166 1 КГ = 116663
1OГР = 347 100 ГР = 347
"PУБ. РУБ.
100 ГР= 324" 100 ГР = 324 
1OOГP 130" 100 ГР = 130
1OГP=89~ 100 ГР = 8900
1OOГP= 431" 100 ГР = 431 
1O ГP=369" 100 ГР = 369 
10O ГР=299" 100 ГР = 299
1O0ГP = 100 100ГР = 100
10OГР= 218" 100 ГР = 218
1 KГ= 831* 1 КГ = 831
100 ГР =399* 100 ГР = 399
PУБ. РУБ.
РYБ. РУБ.
100 ГР =879 100 ГР = 87980
РУБ. РУБ.
1O0 ГР= 273 100 ГР = 273
100ГP = 424" 100 ГР = 424
100 ГР = 106" 100 ГР = 106 
1KГ= 679* 1 КГ = 679
PУБ. РУБ.
PУБ. РУБ.
РУБ. РУБ.
РУБ. РУБ.
PУБ. РУБ.
PУБ. РУБ.
1KГ =1319" 1 КГ =1319
PУБ. РУБ.
РУБ. РУБ.
РУБ. РУБ.
1OOГР=46 100ГР = 46
1KГ=1049" 1 КГ = 1049
РУБ. РУБ.
100ГР= 329" 100 ГР = 329
PУБ. РУБ.
100ГP = 245" 100 ГР = 245 
100 ГP = 232" 100 ГР = 232
1KГ=999* 1 КГ = 99995
1O0ГР=329" 100 ГР = 32940
PУБ. РУБ.
PУБ. РУБ.
РУБ. РУБ.
PУБ. РУБ.
РУБ. РУБ.
100 ГР=329 100 ГР = 32940
1KГ=499" 1 КГ = 49997
PYБ. РУБ.
1D0 ГР = 117" 100 ГР = 117 
РУБ. РУБ.
РУБ. РУБ.


1O0 ГP= L07 100 ГР = 10713
PYБ. РУБ.
1OOP=94" 100 ГР = 9411
PУБ. РУБ.
PУБ. РУБ.
1OOГP=127" 100 ГР = 12777
PУБ. РУБ.
1OO ГP=69.99 100ГР = 69.99
1OOГP=73.50 100 ГР = 73.50
1OO Р=65" 100 ГР = 65
1OOГP=81 100 ГР = 8110
1OO ГР=88" 100 ГР = 88
10OГP= 1O6.95 100 ГР = 106.95
100 ГPМ 94.04 100 ГР = 94.04
1OO ГP=5З.57 100 ГР = 53.57
1OO ГP=5З.57 100 ГР = 53.57
1OO ГР =L6.12 100 ГР = 16.12
"РУБ. РУБ.
РУБ. РУБ.
1OO ГP=2O.37 100 ГР = 20.37
IOO ГРM 14Z.46 100 ГР = 147.46
PУБ. РУБ.
PУБ. РУБ.
РYБ. РУБ.
1OO ГP=69.99 100 ГР =69.99
OГ=57.99 100 ГР = 57.99
РYБ. РУБ.
РУБ. РУБ.
РУБ. РУБ.
1KГ=259" 1КГ = 259
PУБ. РУБ.
PУБ. РУБ.
1KГ = 157" 1 КГ =157
YБ. РУБ.
РУБ. РУБ.
PУБ. РУБ.
1KГ= 300" 1 КГ = 300
PУБ. РУБ.
PYБ РУБ
1KГ= 255" 1 КГ =255
1KГ= 228" 1 КГ = 228
1KГ= 157" 1КГ =157
РУБ. РУБ.
PYБ. РУБ.
РУБ. РУБ.
PУБ. РУБ.
РУБ. РУБ.
PУБ РУБ.
D0 ГP + 99" 100 ГР = 99
PУБ. РУБ.
10O ГР= 89" 100ГР = 8999
*PУБ. РУБ.
PУБ. РУБ.
PУБ. РУБ.
РУБ. РУБ.
OШТ=91 100 ШТ = 91
PУБ. РУБ.
РУБ РУБ.
PYБ. РУБ.
PУБ. РУБ.
PУБ. РУБ.
PУБ. РУБ.
JO

1OORP=22.46 100 ГР = 22.46
PУБ. РУБ.
OOP-З5.29 100 ГР = 35.29
PУБ. РУБ.
1OO ГP=38.36 100 ГР = 38.36
1OO ГP=ЗO.55 100 ГР = 30.55
1OO ГP= 1O2.28 100 ГР = 102.28
1OO ГP=37.2O 100 ГР = 37.20
1OO ГP=36 100 ГР = 3606
1OOГP=55.33 100 ГР = 55.33
PУБ. РУБ.
1OOГP=59.99 100 ГР = 59.99
1OORP=59.99 100 ГР = 59.99
РУБ. РУБ.
PУБ. РУБ.
РУБ РУБ.
1OOГP=59.99 100 ГР = 59.99
10O ГP=178" 100 ГР = 17899
1OГP.= L78" 100 ГР = 17899
1OOГP=L45" 100 ГР = 145
1OO ГP=239.99 100 ГР = 239.99
100 ГР=236* 100 ГР = 23699
PУБ. РУБ.
100 ГP= 107 100 ГР = 10713
1OO ГP= 186.99 100 ГР = 186.99
1OО P =92.OО 100 ГР = 92.00
1OO ГP=81" 100 ГР = 8199
10O ГP=67 100 ГР = 6705
100ГP= 57" 100 ГР = 57
1KГ= 366" 1 КГ = 36694
1OOГР=92" 100 ГР = 9221
1OO ГP=83" 100 ГР = 8350
1OOГP=114.99 100 ГР = 114.99
1OOГP=1O4.99 100 ГР = 104.99
1ОORP=9O.55 100 ГР = 90.55
РYБ. РУБ.
JOO ГP=61 100 ГР = 6110
1O0ГP=61 100 ГР = 6110
PУБ. РУБ.
1OO ГP=9O.OO 100 ГР = 90.00
10 ГР - 109* 100 ГР = 109
PУБ. РУБ.
1OO RP=299" 100 ГР = 29998
10O ГP=69" 100 ГР = 6999

100 ГР =266* 100 ГР = 26699
РYБ. РУБ.
1OO ГP=123 100 ГР = 12350
1OO ГP= 1O5 100 ГР = 10500
1OOГP=136.15 100 ГР = 136.15
1KГ=83* 1 КГ = 8358
1KГ=73" 1 КГ = 73
100 ГР=105"* 100 ГР = 105
1KГ= 1199" 1 КГ = 1199
1O0 ГР=66 100 ГР = 66
1O0 ГР=91 100 ГР = 9187
100 ГР=92 100 ГР = 92
100 ГР=93 100 ГР = 9350
1KГ = 84 1 КГ = 8443
100 ГP=83 100 ГР = 8350
1OO ГP=88" 100 ГР = 88
1 KГ =2249* 1 КГ = 224992
1 KГ =86 1 КГ = 8665
РУБ. РУБ.
1KГ =488" 1 КГ = 488
1 KГ = 1422 1 КГ = 1422
1Л=139* 1 Л = 13945
1KГ -87 1 КГ = 8700
РУБ. РУБ.
РУБ. РУБ.
1 KГ=1162 1 КГ = 116238
1OO RP=64 100 ГР = 64
1Д = I65" 1 Л = 165
1KГ=79 1 КГ = 7978
100ГР= 22 100 ГР = 22
10OГP=27 100 ГР = 2701
1KГ=231 1 КГ = 23155
1KГ=296* 1 КГ = 296
1КГ=196* 1 КГ = 19684
1KГ=2O4" 1 КГ = 204
1OOГР=21* 100 ГР = 21
1KГ=203 1 КГ = 203
РУБ. РУБ.
РУБ. РУБ.
1KГ= 89" 1 КГ = 89
1KГ =295" 1 КГ = 295
PУБ. РУБ.
1KГ =134" 1 КГ = 134
1KГ=289 1 КГ = 289
1KГ=216* 1 КГ = 21695
РУБ. РУБ.
1KГ= 123* 1 КГ = 123
PУБ. РУБ.
РУБ. РУБ.
1KГ= 1399" 1 КГ = 1399
100 ГP=74" 

ЧEРНЫЙ 424Г ЧЕРНЫЙ 424Г
РУБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
PУБ. РУБ.
1OOГР= 326" 100 ГР = 326
1OГР=4" 100 ГР = 4499
4" 100 ГР = 4000
10O ГP= 135" 100 ГР = 135
1OO ГP= 135" 100 ГР = 13599
1OOГP=135" 100 ГР = 135
1ОD FР*69" 100 ГР = 69
1OO RP=69" 100 ГР = 69
1O0 ГP -69" 100 ГР = 69
OOГP=119" 100 ГР = 119
1ЙORP=61" 100 ГР = 61
PУБ. РУБ.
OOГP=31" 100 ГР = 3111
PУБ. РУБ.
PУБ. РУБ.
1KГ= 879 1 КГ = 87996
РУБ. РУБ.
100 ГР = 130 100 ГР = 13020
Ч0 ГР-299" 100 ГР = 299
100ГР = 130" 100 ГР = 130
1O0 ГР=7O 100 ГР = 7000
OГР= 130 100 ГР = 13020
0ГP* 15" 100 ГР = 115
1 KГ= 1266" 1 КГ = 1266
1OO ГР=73 100 ГР = 73
-1O% -10%
 100 ГР = 166
100ГР= 1O6" 100 ГР =10666
1KГ=259" 1 КГ = 259
1KГ=255" 1 КГ =25590
1KГ= 127" 1 КГ = 12745
D100 ГP=11 100 ГР = 11160
10OГР= 135" 100 ГР = 135
OГР ~ 1O5" 100 ГР = 105
100ГР = 120 100 ГР = 12000
1OОГP = 72" 100 ГР = 7217
100 ГP=69 100 ГР = 6956
100 ГР = 142" 100 ГР = 142
1O0 ГР =149" 100 ГР =14999
1OOГP=166" 100 ГР = 166
O0 RP= 72" 100 ГР = 72
-17% -17%
10O ШT = 91 1

In [9]:
print('Levinstain distance: ', lev_dist/len(string_data))
print('Accurancy: ', match/len(string_data))

Levinstain distance:  0.30078332518456635
Accurancy:  0.24486666417324307


In [11]:
print('Levinstain distance: ', lev_dist/len(string_data))
print('Accurancy: ', match/len(string_data))

0.3031367396297006
0.23723678797172457


In [9]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.2986036127712828
0.25359988031566744


In [9]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.3310746717491847
0.22865317724501627


In [10]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.33543471234916805
0.22923289823091597


In [9]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.3545878951220188
0.18917604817294387


In [10]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.37406990236579035
0.16082582189475259


In [14]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.36911594371700374
0.15770280884168006
